## The Numbers Data
[the numbers](https://www.the-numbers.com/movie/budgets/all)

In [1]:
MAX_PAGE = 5701
BASE_URL = 'https://www.the-numbers.com/movie/budgets/all'

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import sqlite3
import matplotlib.pyplot as plt

In [3]:
response = requests.get(BASE_URL)
response.status_code

200

In [4]:
soup = BeautifulSoup(response.content, 'html.parser')

In [5]:
tables = soup.find_all('table')

In [6]:
table = tables[0]

In [7]:
len(table.find_all('th'))

6

In [8]:
table_headers = [th.text for th in table.find_all('th')]
table_headers[0] = 'index'
table_headers

['index',
 'Release Date',
 'Movie',
 'Production Budget',
 'Domestic Gross',
 'Worldwide Gross']

In [9]:
table_rows = [tr for tr in table.find_all('tr')[1:]]
table_rows[0]

<tr><td class="data">1</td>
<td><a href="/box-office-chart/daily/2009/12/18">Dec 18, 2009</a></td>
<td><b><a href="/movie/Avatar#tab=summary">Avatar</a></b></td>
<td class="data">$425,000,000</td>
<td class="data">$760,507,625</td>
<td class="data">$2,776,345,279</td>
</tr>

In [10]:
def process_the_numbers_row(table_rows):
    data = []
    for row in table_rows:
        data_row = []
        # append index
        data_row.append(row.find('td').text)
        # append Release Date
        data_row.append(row.find('a').text)
        # append Movie Title
        data_row.append(row.find_all('a')[1].text)
        # append Production Budget
        data_row.append(row.find_all('td')[3].text)
        # append Domestic Gross
        data_row.append(row.find_all('td')[4].text)
        # append Worldwide Gross
        data_row.append(row.find_all('td')[5].text)
        # append link
        data_row.append(row.find('a')['href'])
        #append data_row to data
        data.append(data_row)
    
    return data

In [11]:
table_headers[2] = 'Movie Title'
table_headers.append('Link')

In [12]:
len(data)

NameError: name 'data' is not defined

In [13]:
def process_the_numbers_page(soup):
    """
    Take in a page from the-numbers.com
    """
    tables = soup.find_all('table')
    table = tables[0]
    table_rows = [tr for tr in table.find_all('tr')[1:]]
    page_data = process_the_numbers_row(table_rows)
    
    return page_data

In [14]:
BASE_URL+'/'+'i'

'https://www.the-numbers.com/movie/budgets/all/i'

In [15]:
import time
the_movies_data = []
for i in range(1, MAX_PAGE+1, 100):
    print(f'movies {i} to {i+99}')
    response = requests.get(BASE_URL+'/'+str(i))
    soup = BeautifulSoup(response.content, 'html.parser')
    the_movies_data.extend(process_the_numbers_page(soup))
    time.sleep(1)

movies 1 to 100
movies 101 to 200
movies 201 to 300
movies 301 to 400
movies 401 to 500
movies 501 to 600
movies 601 to 700
movies 701 to 800
movies 801 to 900
movies 901 to 1000
movies 1001 to 1100
movies 1101 to 1200
movies 1201 to 1300
movies 1301 to 1400
movies 1401 to 1500
movies 1501 to 1600
movies 1601 to 1700
movies 1701 to 1800
movies 1801 to 1900
movies 1901 to 2000
movies 2001 to 2100
movies 2101 to 2200
movies 2201 to 2300
movies 2301 to 2400
movies 2401 to 2500
movies 2501 to 2600
movies 2601 to 2700
movies 2701 to 2800
movies 2801 to 2900
movies 2901 to 3000
movies 3001 to 3100
movies 3101 to 3200
movies 3201 to 3300
movies 3301 to 3400
movies 3401 to 3500
movies 3501 to 3600
movies 3601 to 3700
movies 3701 to 3800
movies 3801 to 3900
movies 3901 to 4000
movies 4001 to 4100
movies 4101 to 4200
movies 4201 to 4300
movies 4301 to 4400
movies 4401 to 4500
movies 4501 to 4600
movies 4601 to 4700
movies 4701 to 4800
movies 4801 to 4900
movies 4901 to 5000
movies 5001 to 5100
m

In [16]:
# Fix the headers from before
table_headers[0] = 'i'

In [17]:
conn = sqlite3.connect(':memory:')
cur = conn.cursor()
column_names = ','.join([header.replace(' ','') + ' varchar\n' for header in table_headers])

In [18]:
column_names

'i varchar\n,ReleaseDate varchar\n,MovieTitle varchar\n,ProductionBudget varchar\n,DomesticGross varchar\n,WorldwideGross varchar\n,Link varchar\n'

In [43]:
sql = """CREATE TABLE movies_5 (i TEXT,ReleaseDate varchar,MovieTitle varchar,ProductionBudget varchar,DomesticGross varchar,WorldwideGross varchar,Link varchar)"""
cur.execute(sql)#.format(col_names=column_names))

In [39]:
insert_sql = """INSERT INTO movies_5 VALUES (?,?,?,?,?,?,?);"""

for d in the_movies_data:
    cur.execute(insert_sql, d)

In [44]:
cur.execute("""
SELECT *
FROM movies_5""")

print(cur.description)
df = pd.DataFrame(cur.fetchall())

(('i', None, None, None, None, None, None), ('ReleaseDate', None, None, None, None, None, None), ('MovieTitle', None, None, None, None, None, None), ('ProductionBudget', None, None, None, None, None, None), ('DomesticGross', None, None, None, None, None, None), ('WorldwideGross', None, None, None, None, None, None), ('Link', None, None, None, None, None, None))


In [45]:
df.head()

""
